# Computing Magnetization With The Suzuki-Trotter Approximation

A key application for quantum computers is the simulation of quantum systems. This tutorial will demonstrate a CUDA-Q implementation of a Suzuki-Trotter simulation of a Heisenberg Model Hamiltonian to compute the magnetization of a spin chain.

This example takes advantage of CUDA-Q's state handling abilities to perform the recursive Trotter simulations. 

### Problem Setup

In [1]:
import cudaq
import time
import numpy as np
from typing import List

The goal of this problem is to estimate the time evolution of an initial quantum state, governed by a Hamiltonian, and then compute the average magnetization fo the final state.  The time evolution can be approximated using the Trotter method:

$$  e^{-iHt} \approx \prod_{n=0}^N e^{\frac{-iHt}{n}}$$

The Heisenberg Hamiltonian is defined below, parameterized by these predefined constants: $g$, $Jx$, $Jy$, and $\omega = 2\pi$.   The time step $dt$ is selected for this problem as well as `n_steps` and `n_spins` ($N$), determining the size of the simulation.  The default setup considers 11 spins and 10 steps, which can easily be simulated on a CPU.  If you have access to at least one GPU, the problem can be increased to around 25 steps and 100 time steps.


$$ H = \sum_{j=1}^{N}(J_x\sigma^x_j\sigma^x_{j+1} + J_x\sigma^y_j\sigma^y_{j+1} + J_x\sigma^z_j\sigma^z_{j+1} + \cos(\omega * t)\sigma^x_j   )$$


In [2]:
g = 1.0
Jx = 1.0
Jy = 1.0
Jz = g
dt = 0.05
n_steps = 10
n_spins = 11
omega = 2 * np.pi


def heisenbergModelHam(t: float) -> cudaq.SpinOperator:
    tdOp = cudaq.SpinOperator.empty()
    for i in range(0, n_spins - 1):
        tdOp += (Jx * cudaq.spin.x(i) * cudaq.spin.x(i + 1))
        tdOp += (Jy * cudaq.spin.y(i) * cudaq.spin.y(i + 1))
        tdOp += (Jz * cudaq.spin.z(i) * cudaq.spin.z(i + 1))
    for i in range(0, n_spins):
        tdOp += (np.cos(omega * t) * cudaq.spin.x(i))
    return tdOp

Next, two CUDA-Q kernels are defined. The first, `getInitState`, prepares an initial state where an $X$ gate is applied to the first and then every other qubit to initialize a chain of alternating spins. 

In [3]:
@cudaq.kernel
def getInitState(numSpins: int):
    q = cudaq.qvector(numSpins)
    for qId in range(0, numSpins, 2):
        x(q[qId])

The second, `trotter`, performs a single Trotter step given some provided initial state and then returns the resulting state.  Two notes should be made about this kernel. 
1. It takes advantage of CUDA-Q's state handling abilities, allowing each step to proceed from the previous, still stored in GPU memory during the simulation.  This provides a significant speedup which is demonstrated in the plot at the end of this tutorial.
2. This kernel takes advantage of the `pauli_word` object which allows a list of Pauli words and their coefficients to be passed into the kernel and applied as an exponentiated matrix operation using `exp_pauli`. 

In [4]:
@cudaq.kernel
def trotter(state: cudaq.State, coefficients: List[complex],
            words: List[cudaq.pauli_word], dt: float):
    q = cudaq.qvector(state)
    for i in range(len(coefficients)):
        exp_pauli(coefficients[i].real * dt, q, words[i])

The functions below are used to strip the Hamiltonian spin operator into a list of coefficients and Pauli words.

In [5]:
def termCoefficients(op: cudaq.SpinOperator) -> List[complex]:
    result = []
    for term in op:
        result.append(term.get_coefficient())
    return result


def termWords(op: cudaq.SpinOperator) -> List[str]:
    # Our kernel uses these words to apply exp_pauli to the entire state.
    # we hence ensure that each pauli word covers the entire space.
    n_spins = op.get_qubit_count()
    result = []
    for term in op:
        result.append(term.get_pauli_word(n_spins))
    return result

Finally, a second spin operator is defined which will be used to compute the expectation value corresponding to the average magnetization.

In [6]:
average_magnetization = cudaq.SpinOperator.empty()
for i in range(0, n_spins):
    average_magnetization += ((1.0 / n_spins) * cudaq.spin.z(i))

### Running the Simulation

Before looping through the Trotter steps, the initial state is constructed.

In [7]:
state = cudaq.get_state(getInitState, n_spins)

Next, the time steps are looped through. At each step, the time dependent Hamiltonian is defined. This Hamiltonian is used to construct the Trotter kernel for that time step, which is then used to compute the average magnetization expectation value.  The state is saved, and used as the initial state for the next time step. 

In [8]:
results = []
times = []
for i in range(0, n_steps):
    start_time = time.time()
    ham = heisenbergModelHam(i * dt)
    coefficients = termCoefficients(ham)
    words = termWords(ham)
    magnetization_exp_val = cudaq.observe(trotter, average_magnetization, state,
                                          coefficients, words, dt)
    result = magnetization_exp_val.expectation()
    results.append(result)
    state = cudaq.get_state(trotter, state, coefficients, words, dt)
    stepTime = time.time() - start_time
    times.append(stepTime)
    print(f"Step {i}: time [s]: {stepTime}, result: {result}")

print(f"Step times: {times}")
print(f"Results: {results}")

Step 0: time [s]: 1.5979282855987549, result: -0.09045492411618333


Step 1: time [s]: 1.2472047805786133, result: -0.0891842902112794


Step 2: time [s]: 1.1704421043395996, result: -0.08746828784221615
Step 3: time [s]: 1.1461787223815918, result: -0.08586188047766233


Step 4: time [s]: 1.248096227645874, result: -0.08489801006386904


Step 5: time [s]: 1.2154357433319092, result: -0.08489801006386798


Step 6: time [s]: 1.2587661743164062, result: -0.08586188047765986


Step 7: time [s]: 1.2403604984283447, result: -0.08746828784221186


Step 8: time [s]: 1.2469470500946045, result: -0.0891842902112737


Step 9: time [s]: 1.255791187286377, result: -0.09045492411617581
Step times: [1.5979282855987549, 1.2472047805786133, 1.1704421043395996, 1.1461787223815918, 1.248096227645874, 1.2154357433319092, 1.2587661743164062, 1.2403604984283447, 1.2469470500946045, 1.255791187286377]
Results: [-0.09045492411618333, -0.0891842902112794, -0.08746828784221615, -0.08586188047766233, -0.08489801006386904, -0.08489801006386798, -0.08586188047765986, -0.08746828784221186, -0.0891842902112737, -0.09045492411617581]


CUDA-Q's state handling capabilities provide a massive performance boost for this algorithm. Rather than resimulate all previous operation at any given time step, saving the previous state in GPU memory allows completion of the simulation with fewer operations. The figure below demonstrates the 24X speedup realized by a 100 step Trotter simulation.

![Htest](./images/statehandle.png)
